<a href="https://colab.research.google.com/github/sarrow04/note1000/blob/main/preprocessing_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import io
import plotly.express as px
import numpy as np

# --- Streamlitアプリの基本設定 ---
st.set_page_config(page_title="データ前処理サポーター", page_icon="🛠️", layout="wide")
st.title("🛠️ データ前処理サポーター")
st.write("CSVやExcelファイルをアップロードするだけで、データの健康診断とクリーニングができます。")

# --- Session Stateの初期化 ---
if 'df' not in st.session_state: st.session_state.df = None
if 'original_df' not in st.session_state: st.session_state.original_df = None

# --- サイドバー ---
with st.sidebar:
    st.header("1. ファイルをアップロード")
    uploaded_file = st.file_uploader("CSVまたはExcelファイルをアップロードしてください", type=['csv', 'xlsx'])
    if uploaded_file is not None:
        if "uploaded_file_name" not in st.session_state or st.session_state.uploaded_file_name != uploaded_file.name:
            try:
                if uploaded_file.name.endswith('.csv'): df = pd.read_csv(uploaded_file)
                else: df = pd.read_excel(uploaded_file)
                st.session_state.uploaded_file_name = uploaded_file.name
                st.session_state.df = df
                st.session_state.original_df = df.copy()
                st.sidebar.success("ファイルが正常に読み込まれました！")
            except Exception as e: st.error(f"ファイルの読み込み中にエラーが発生しました: {e}")

    if st.session_state.df is not None:
        df_sidebar = st.session_state.df
        st.header('2. 前処理ツール')

        # --- 特徴量エンジニアリング ---
        st.subheader('特徴量エンジニアリング')
        numeric_cols = df_sidebar.select_dtypes(include=np.number).columns.tolist()
        operation = st.selectbox('実行したい操作を選択', ['---', '列の合計', '列の積', '列の差', '列の商'])
        if operation != '---':
            if operation in ['列の差', '列の商']:
                cols_to_operate = st.multiselect(f'「{operation}」を計算する数値列を2つ選択', numeric_cols, max_selections=2)
            else:
                cols_to_operate = st.multiselect(f'「{operation}」を計算する数値列を2つ以上選択', numeric_cols)
            new_col_name = st.text_input('新しい列の名前を入力してください', f'new_{operation}')
            if st.button(f'{operation}を実行'):
                if len(cols_to_operate) >= 2 and new_col_name:
                    try:
                        temp_df = st.session_state.df.copy()
                        if operation == '列の合計': temp_df[new_col_name] = temp_df[cols_to_operate].sum(axis=1)
                        elif operation == '列の積': temp_df[new_col_name] = temp_df[cols_to_operate].prod(axis=1)
                        elif operation == '列の差': temp_df[new_col_name] = temp_df[cols_to_operate[0]] - temp_df[cols_to_operate[1]]
                        elif operation == '列の商':
                            if (temp_df[cols_to_operate[1]] == 0).any(): st.error("エラー: 割る数に0が含まれています。")
                            else: temp_df[new_col_name] = temp_df[cols_to_operate[0]] / temp_df[cols_to_operate[1]]
                        st.session_state.df = temp_df
                        st.success(f"新しい列 '{new_col_name}' を作成しました。"); st.rerun()
                    except Exception as e: st.error(f"計算中にエラーが発生しました: {e}")
                else: st.warning("列と新しい列名を確認してください。")

        # --- リセット機能 ---
        if st.button("最初の状態に戻す"):
            st.session_state.df = st.session_state.original_df.copy()
            st.info("データが最初の状態にリセットされました。")
            st.rerun()

# --- メイン画面 ---
if st.session_state.df is not None:
    df_main = st.session_state.df

    st.header("🩺 2. データの健康診断")
    tab1, tab2, tab3 = st.tabs(["基本情報とデータプレビュー", "欠損値の状況", "統計量の確認"])
    with tab1:
        # ▼▼▼ ここの行末に `")` を追加しました ▼▼▼
        st.subheader("基本情報"); st.markdown(f"**行数:** {df_main.shape[0]} 行, **列数:** {df_main.shape[1]} 列")
        st.subheader("データプレビュー"); st.dataframe(df_main.head())
    with tab2:
        st.subheader("各列の欠損値の数"); missing_values = df_main.isnull().sum(); st.dataframe(missing_values[missing_values > 0].sort_values(ascending=False).rename("欠損数"))
    with tab3:
        st.subheader("各列の統計量"); st.dataframe(df_main.describe(include='all'))

    st.header("🧹 3. データ全体のクリーニング")
    st.subheader("列の一括削除")
    columns_to_drop = st.multiselect('不要な列を複数選択できます。', df_main.columns)
    if st.button("選択した列を削除する"):
        if columns_to_drop:
            st.session_state.df = df_main.drop(columns=columns_to_drop)
            st.success("選択された列を削除しました。"); st.rerun()
        else: st.warning("削除する列が選択されていません。")

    num_duplicates = df_main.duplicated().sum()
    if num_duplicates > 0:
        st.write(f"データセット全体に **{num_duplicates}** 件の重複行があります。")
        if st.button("重複行をすべて削除する"):
            st.session_state.df = df_main.drop_duplicates()
            st.success("重複行を削除しました。"); st.rerun()

    st.header("💊 4. 列ごとの対話型クリーニング")
    st.write("処理したい列を選択し、アクションを実行してください。")
    selected_column = st.selectbox("処理対象の列を選択してください", df_main.columns)
    if selected_column:
        col_type = df_main[selected_column].dtype; missing_count = df_main[selected_column].isnull().sum(); st.write(f"選択中の列: **{selected_column}** (データ型: {col_type}, 欠損値: {missing_count}個)")
        st.subheader(f"「{selected_column}」列へのアクション")
        if missing_count > 0:
            with st.expander("欠損値の処理"):
                fill_method = st.radio("欠損値をどうしますか？", ("平均値で埋める", "中央値で埋める", "最頻値で埋める", "指定した値で埋める", "行ごと削除する"), key=f"fill_{selected_column}")
                fill_value = None
                if fill_method == "指定した値で埋める": fill_value = st.text_input("埋める値を入力してください")
                if st.button("欠損値処理を実行", key=f"btn_fill_{selected_column}"):
                    df_copy = df_main.copy()
                    if fill_method == "平均値で埋める": df_copy[selected_column].fillna(df_copy[selected_column].mean(), inplace=True)
                    elif fill_method == "中央値で埋める": df_copy[selected_column].fillna(df_copy[selected_column].median(), inplace=True)
                    elif fill_method == "最頻値で埋める": df_copy[selected_column].fillna(df_copy[selected_column].mode()[0], inplace=True)
                    elif fill_method == "指定した値で埋める" and fill_value: df_copy[selected_column].fillna(fill_value, inplace=True)
                    elif fill_method == "行ごと削除する": df_copy.dropna(subset=[selected_column], inplace=True)
                    st.session_state.df = df_copy; st.success(f"「{selected_column}」列の欠損値処理が完了しました。"); st.rerun()

        with st.expander("データ型の変換"):
            new_type = st.selectbox("変換したいデータ型を選択", ["---", "数値 (int)", "数値 (float)", "文字列 (str)", "日付 (datetime)"], key=f"type_{selected_column}")
            if st.button("データ型を変換", key=f"btn_type_{selected_column}"):
                if new_type != "---":
                    try:
                        df_copy = df_main.copy()
                        temp_series = df_copy[selected_column]
                        pre_missing = temp_series.isnull().sum()
                        if new_type in ["数値 (int)", "数値 (float)"]:
                            temp_series = temp_series.astype(str).str.replace(r'[^\d.]', '', regex=True).replace('', pd.NA)
                            temp_series = pd.to_numeric(temp_series, errors='coerce')
                            if new_type == "数値 (int)": temp_series = temp_series.astype('Int64')
                        elif new_type == "文字列 (str)": temp_series = temp_series.astype(str)
                        elif new_type == "日付 (datetime)":
                            def clean_date_str(s):
                                if not isinstance(s, str): return s
                                if len(s) == 10 and s.isdigit(): return f"{s[:4]}-{s[6:8]}-{s[8:]}"
                                s = s.replace('年', '-').replace('月', '-').replace('日', '')
                                return s[:-1] if s.endswith('-') else s
                            temp_series = temp_series.apply(clean_date_str)
                            temp_series = pd.to_datetime(temp_series, errors='coerce')
                        df_copy[selected_column] = temp_series
                        post_missing = df_copy[selected_column].isnull().sum()
                        st.session_state.df = df_copy; st.success(f"「{selected_column}」列を{new_type}型に変換しました。")
                        if post_missing > pre_missing: st.warning(f"{post_missing - pre_missing}個のデータが変換に失敗し、欠損値になりました。")
                        st.rerun()
                    except Exception as e: st.error(f"変換に失敗しました: {e}")

        if pd.api.types.is_string_dtype(df_main[selected_column]):
            with st.expander("ワンホットエンコーディング"):
                if st.button("ワンホットエンコーディングを実行", key=f"btn_ohe_{selected_column}"):
                    dummies = pd.get_dummies(df_main[selected_column], prefix=selected_column)
                    df_copy = pd.concat([df_main.drop(selected_column, axis=1), dummies], axis=1)
                    st.session_state.df = df_copy; st.success(f"ワンホットエンコーディングを実行しました。"); st.rerun()

    st.header("✅ 5. 処理済みデータのダウンロード")
    @st.cache_data
    def convert_df_to_csv(df): return df.to_csv(index=False).encode('utf-8')
    csv = convert_df_to_csv(st.session_state.df); st.download_button(label="整形済みデータをCSVでダウンロード", data=csv, file_name='cleaned_data.csv', mime='text/csv')
else:
    st.info("サイドバーからファイル（CSVまたはExcel）をアップロードして分析を開始してください。")